## **Tone Enhancement: Informal to formal**

# *Installing required packages*



In [1]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install tensorflow
!pip install transformers
!pip install symspellpy
!pip install nltk
!pip install symspellpy
!pip install keras

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.3 MB/s eta 0:00:00
  Created wheel for editdistpy: filename=editdistpy-0.1.4-cp310-cp310-linux_x86_64.whl size=187485 sha256=4e7074f0722218686f260a425d49bdf05c42d62088eb6b3c0654709ed719612d
  Stored in directory: /root/.cache/pip/wheels/4c/0f

# *Importing packages*

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
import string
import re
from symspellpy import SymSpell, Verbosity

tf.random.set_seed(42)

data = pd.read_csv('/content/with_augmentation_2.csv')

In [4]:
data.head()

,informal_text_inp,normal_text_inp,normal_text_out
0,<Do you want me to hold seat for you operating...,<Do you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?>
1,<Yeap. You reaching? We ordered some Durian pa...,<Yeap. You reaching? We ordered some Durian pa...,Yeap. You reaching? We ordered some Durian pas...
2,<They turn more expensive already. Mine be com...,<They become more expensive already. Mine is l...,They become more expensive already. Mine is li...
3,<1 ' m Thai. What do you make?>,<I'm Thai. What do you do?,I'm Thai. What do you do?>
4,<How do you do! How did your week go? Haven ' ...,<Hi! How did your week go? Haven't heard from ...,Hi! How did your week go? Haven't heard from y...


# *Pre-processing dataframe*

In [5]:
data.isnull().sum()

informal_text_inp    0
normal_text_inp      0
normal_text_out      0
dtype: int64

In [6]:
data.fillna('', inplace=True)

In [10]:
# removing punctuations and special character
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [8]:
from symspellpy import SymSpell, Verbosity
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
def correct_spelling_with_symspell(text):
suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
corrected_text = suggestions[0].term
return corrected_text


# *Initializing tokenizer and preparing it as model input*

In [9]:
X = data['informal_text_inp'].astype(str) # list of informal sentences
Y = data['normal_text_inp'].astype(str) # list of formal/normal sentences

# cleaning and normalizing the dataset
cleaned_X = X.apply(remove_punctuation).apply(remove_special_characters)
cleaned_Y = Y.apply(remove_punctuation).apply(remove_special_characters)
X_lower_and_cleaned = cleaned_X.str.lower()
Y_lower_and_cleaned = cleaned_Y.str.lower()

X_corrected = X_lower_and_cleaned.apply(correct_spelling_with_symspell)
Y_corrected = Y_lower_and_cleaned.apply(correct_spelling_with_symspell)

# calling and initializing the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# making input_ids and attention_mask
X_sequence = tokenizer(X_corrected.tolist(), padding=True, truncation=True, return_tensors='tf')
Y_sequence = tokenizer(Y_corrected.tolist(), padding=True, truncation=True, return_tensors='tf')

X_input_ids = X_sequence['input_ids'].numpy()
Y_input_ids = Y_sequence['input_ids'].numpy()
X_attention_mask = X_sequence['attention_mask'].numpy()
Y_attention_mask = Y_sequence['attention_mask'].numpy()

# making train , test and validation inputs and attention masks of model
X_train, X_test, Y_train, Y_test = train_test_split(X_input_ids, Y_input_ids, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(X_train,Y_train,test_size = 0.1, random_state = 42)
X_train_attention_mask, X_test_attention_mask, Y_train_attention_mask, Y_test_attention_mask = train_test_split(X_attention_mask, Y_attention_mask, test_size=0.1, random_state=42)
x_train_attention_mask, x_val_attention_mask, y_train_attention_mask, y_val_attention_mask = train_test_split(X_train_attention_mask, Y_train_attention_mask, test_size=0.1, random_state=42)

# making it in form of dataset which model can process on
model_input = {
    'input_ids': x_train,
    'attention_mask': x_train_attention_mask,
    'labels': y_train
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Initializing the model and compiling it**

In [11]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy() # calling a mirror strategy i.e., we can run code on multiple GPus

with strategy.scope():
    model_name = 't5-small'
    model = TFT5ForConditionalGeneration.from_pretrained(model_name) # initializing a model t5 - small
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.SUM  # defing a loss function which sums up the losses
    )

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy']) # eval metrics = 'accuracy'

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# **Making 3 datasets : train, test , and validation**

In [13]:
batch_size = 16

# spliting the dataset into train, test, and validation

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': model_input['input_ids'],
        'attention_mask': model_input['attention_mask'],
        'labels': model_input['labels']
    }
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_test,
        'attention_mask': X_test_attention_mask,
        'labels': Y_test
    }
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': X_test,
        'attention_mask': X_test_attention_mask,
        'labels': Y_test
    }
)).batch(batch_size)

# **Training**

In [14]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=4
)

Epoch 1/4
608/608 [==============================] - 251s 318ms/step - loss: 446.2502 - accuracy: 0.9377 - val_loss: 256.8667 - val_accuracy: 0.9587
Epoch 2/4
608/608 [==============================] - 177s 291ms/step - loss: 233.5814 - accuracy: 0.9610 - val_loss: 200.9270 - val_accuracy: 0.9668
Epoch 3/4
608/608 [==============================] - 176s 289ms/step - loss: 169.9546 - accuracy: 0.9695 - val_loss: 176.7830 - val_accuracy: 0.9711
Epoch 4/4
608/608 [==============================] - 177s 291ms/step - loss: 125.9798 - accuracy: 0.9759 - val_loss: 161.4169 - val_accuracy: 0.9742


# **testing**

In [15]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}")

75/75 [==============================] - 10s 136ms/step - loss: 161.4169 - accuracy: 0.9742
Loss: 161.41690063476562, Accuracy: 0.9742434024810791


# **generate output**

In [20]:
def generate_output(input_text, tokenizer, model, max_length=50):
    input_ids = tokenizer.encode(input_text, return_tensors='tf', padding=True, truncation=True)
    outputs = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output

user_input = input("Enter an informal text to normalize: ")
generated_output = generate_output(user_input, tokenizer, model)
generated_output = generated_output[0].upper() + generated_output[1:]

print(f"Generated normalized text: {generated_output}.")

Enter an informal text to normalize: I will kill him 4 u
Generated normalized text: I will kill him for you.
